# Задание 1.

Обязательная часть
Будем парсить страницу со свежеми новостям на habr.com/ru/all/.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

KEYWORDS = ['python', 'парсинг']

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>

Дополнительная часть (необязательная)

Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.

Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст_статьи>

In [63]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

def parsePosts(keywords, isFull = False):
    res = requests.get('https://habr.com/ru/all/')
    soup = BeautifulSoup(res.text, 'html.parser')
    posts = soup.find_all('div', class_='post__text')

    df = pd.DataFrame()

    for post in posts:
        for keyword in keywords:
            if isFull:
                link = post.parent.parent.find('a', class_='post__title_link').attrs.get('href')
                detail = requests.get(link)
                soup = BeautifulSoup(detail.text, 'html.parser')
                articles = soup.find_all('div', class_='post__text')
                index = articles[0].text.lower().find(keyword)
                if index != -1:
                    df = pd.concat([df, pd.DataFrame([getRow(post, articles[0])])]) 
            else:
                index = post.text.lower().find(keyword)
                if index != -1:
                    df = pd.concat([df, pd.DataFrame([getRow(post)])])
    return df

def getRow(post, article = False):
    parent = post.parent.parent
    date = parent.find('span', class_='post__time')
    link = parent.find('a', class_='post__title_link').attrs.get('href')
    title = parent.find('a', class_='post__title_link').text
    if article:
        return {'date': date.text, 'title': title, 'link': link, 'article': article.text}
    else:
        return {'date': date.text, 'title': title, 'link': link}
    
KEYWORDS = ['python', 'парсинг']
parsePosts(KEYWORDS)

,date,title,link
0,вчера в 22:50,Conways Game of life на Python,https://habr.com/ru/post/532732/


In [64]:
parsePosts(KEYWORDS, True)

,date,title,link,article
0,вчера в 22:50,Conways Game of life на Python,https://habr.com/ru/post/532732/,"Это мой первый пост, где я хочу рассказать про..."
0,вчера в 20:21,"Russian AI Cup 2019. 4 место, почти не умея пр...",https://habr.com/ru/post/532716/,Данная статья рассмотрит процесс моего участия...
0,вчера в 18:44,Визуализация данных по акциям дивидендных арис...,https://habr.com/ru/post/532622/,В этой статье я поделюсь некоторым опытом на ...
0,вчера в 17:15,ALog: плюс один логгер для С++ приложений,https://habr.com/ru/post/532698/,Система логирования ALog первоначально разраба...
0,вчера в 15:57,Как я превратил свой стол с регулировкой высот...,https://habr.com/ru/company/skillfactory/blog/...,"В этой статье я покажу, как я превратил свой с..."
0,вчера в 15:37,Google предложил рейтинг критически важных Ope...,https://habr.com/ru/company/selectel/blog/532680/,"\r\nО многих проектах говорят, что они крайне ..."
0,вчера в 15:22,Написание WinLocker'а на Python,https://habr.com/ru/post/532684/,Внимание!Данная статья была создана в познават...


# Задание 2.
Обязательная часть
Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. Список email-ов задаем переменной в начале кода:
EMAIL = [xxx@x.ru, yyy@y.com]

В итоге должен формироваться датафрейм со столбцами: <почта> - <дата утечки> - <источник утечки> - <описание утечки>

Подсказка: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.

Дополнительная часть (необязательная)
Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте.
Документация к API VK: https://vk.com/dev/methods , вам поможет метод wall.get

GROUP = 'netology'  
TOKEN = УДАЛЯЙТЕ В ВЕРСИИ ДЛЯ ПРОВЕРКИ, НА GITHUB НЕ ВЫКЛАДЫВАТЬ
В итоге должен формироваться датафрейм со столбцами: <дата поста> - <текст поста>

In [90]:
import json

df = pd.DataFrame()
EMAIL = ['xxx@x.ru', 'yyy@y.com']
URL = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'
params = {
    json.dumps({'emailAddresses': ['xxx@x.ru']}),
}
headers = {
    'Content-Type': 'application/json;charset=UTF-8',
    'Vaar-Header-App-Product': 'hackcheck-web-avast',
    'Vaar-Version': '0'
}

for email in EMAIL:
    req = requests.post(URL, data=json.dumps({'emailAddresses': [email]}), headers=headers)

    breaches = req.json()['breaches']
    for breach in breaches:
        row = {'email': email, 'date': breaches[breach]['publishDate'], 'source': breaches[breach]['site'], 'description': breaches[breach]['description'], }
        df = pd.concat([df, pd.DataFrame([row])])

df

,email,date,source,description
0,xxx@x.ru,2019-03-28T00:00:00Z,verifications.io,Big data e-mail verification platform verifica...
0,xxx@x.ru,2020-05-21T00:00:00Z,vk.com,"At some time in 2020, the Russian social netwo..."
0,xxx@x.ru,2017-02-14T00:00:00Z,parapa.mail.ru,"In July and August 2016, two criminals execute..."
0,xxx@x.ru,2016-10-29T00:00:00Z,vk.com,Popular Russian social networking platform VKo...
0,xxx@x.ru,2016-10-21T00:00:00Z,adobe.com,"In October of 2013, criminals penetrated Adobe..."
0,xxx@x.ru,2017-02-14T00:00:00Z,cfire.mail.ru,"In July and August of 2016, two criminals carr..."
0,xxx@x.ru,2017-01-31T00:00:00Z,cdprojektred.com,"In March 2016, CDProjektRed.com.com's forum da..."
0,xxx@x.ru,2016-10-23T00:00:00Z,imesh.com,"In June 2016, a cache of over 51 million user ..."
0,yyy@y.com,2020-01-03T00:00:00Z,azcentral.com,"At an unconfirmed date, online Arizona newspap..."
0,yyy@y.com,2020-05-28T00:00:00Z,wishbone.io,"In January 2020, the online poll website Wishb..."


In [100]:
from datetime import datetime

URL = 'https://api.vk.com/method/wall.get?'
TOKEN = '***'
VERSION = '5.103'
GROUP = 'netology'

df = pd.DataFrame()

params = {
    'access_token': TOKEN,
    'v': VERSION,
    'domain': GROUP,
    'count': 50
}
req = requests.post(URL, params).json()
for item in req['response']['items']:
    row = {'post': item['text'], 'date': datetime.utcfromtimestamp(item['date']).strftime('%Y-%m-%d %H:%M:%S')}
    df = pd.concat([df, pd.DataFrame([row])])
df

,post,date
0,Наш предновогодний digital-марафон продолжаетс...,2020-12-07 10:05:00
0,Чужой пример заразителен 😉\nИстории успеха кру...,2020-12-13 08:19:00
0,В новой афише мероприятий конференции по марке...,2020-12-12 12:56:00
0,"Как упростить работу с таблицами, улучшить диа...",2020-12-12 08:04:01
0,"В эфире #пятничнаябеседка. Здесь мы обсуждаем,...",2020-12-11 14:18:00
0,Поиграем со шрифтами?😅 \n\nРазбираемся с графи...,2020-12-11 07:28:00
0,Учёт доходов и расходов — первый шаг к устране...,2020-12-10 14:48:00
0,*партнёрский пост*\n\nПодводим итоги года с Le...,2020-12-10 11:18:00
0,"Будущее ближе, чем мы думаем 🤖 \n \nВ новом Da...",2020-12-10 07:19:00
0,У нас сегодня Нето-викторина 🤩\n\nМы подготови...,2020-12-09 13:25:03
